In [59]:
import tensorflow as tf
import numpy as np
import timeit

### Tensors

 Tensors are multidimensional arrays

In [28]:
x = tf.constant([[1, 2, 4],
                 [3, 5, 6]])

x

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [29]:
print(f"Tensor x \n {x}")
print(f"Tensor x shape :   {x.shape}")
print(f"Tensor x dtype :   {x.dtype}")

Tensor x 
 [[1 2 4]
 [3 5 6]]
Tensor x shape :   (2, 3)
Tensor x dtype :   <dtype: 'int32'>


In [30]:
y = tf.convert_to_tensor(np.array([[1, 0, 1],[1, 1, 1]]))
y

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 0, 1],
       [1, 1, 1]])>

In [31]:
### Tensor operations

print(f"Transpose of x \n {x @ tf.transpose(x)}")

print(f"Concat x & y \n {tf.concat([x,y], axis=0)}")

print(f"Reduce sum {tf.reduce_sum([1,2,3])}")

Transpose of x 
 [[21 37]
 [37 70]]
Concat x & y 
 [[1 2 4]
 [3 5 6]
 [1 0 1]
 [1 1 1]]
Reduce sum 6


### Variables

Normal tf.Tensor objects are immutable. To store model weights (or other mutable state) in TensorFlow use a `tf.Variable`.

In [37]:
x_var = tf.Variable(x)
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [33]:
### Adding values to varible within the same ref
x_var.assign_add(y)
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[2, 2, 5],
       [4, 6, 7]])>

In [38]:
x_var.assign([[1, 2, 4],
       [3, 5, 6]])
x_var

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 4],
       [3, 5, 6]])>

In [39]:
#Most tensor operations work on variables as expected, although variables cannot be reshaped. 
## its creating tensor not varible

x_var_re = tf.reshape(x_var, [3,2])
x_var_re

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [4, 3],
       [5, 6]])>

### Automatic differentiation
Implementing automatic differentiation (autodiff), which uses calculus to compute gradients. Typically you'll use this to calculate the gradient of a model's error or loss with respect to its weights.

Gradient tapes
TensorFlow provides the `tf.GradientTape` API for automatic differentiation;
Once you've recorded some operations, use `GradientTape.gradient`(target, sources) to calculate the gradient of some target (often a loss) relative to some source (often the model's variables):

In [4]:
x = tf.Variable(2.0)

def fun(x):
    y = x**2+2*x+5
    
    return y

fun(x)

<tf.Tensor: shape=(), dtype=float32, numpy=13.0>

In [ ]:
with tf.GradientTape() as tape:
    y = fun(x)

g_x = tape.gradient(y, x)  # g(x) = dy/dx  = 2x+2
g_x

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [6]:
## Gradient with two varibles

w = tf.Variable(tf.random.normal((3,2)), name='w')
b = tf.Variable(tf.zeros(2), name='b')

x=[[1., 2., 3.]]

In [7]:
with tf.GradientTape() as tape:
    y = x @ w +b
    loss = tf.reduce_mean(y**2)

In [8]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

In [11]:
w, b

(<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.49473074, -2.23581   ],
        [-1.1306745 ,  0.03210247],
        [-1.1583433 , -1.6996672 ]], dtype=float32)>,
 <tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>)

In [10]:
dl_dw, dl_db

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ -5.241648,  -7.270607],
        [-10.483296, -14.541214],
        [-15.724945, -21.811821]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-5.241648, -7.270607], dtype=float32)>)

In [ ]:
###Gradients with respect to a model

layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

y = layer(x)

y,layer    

(<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.       , 3.0652003]], dtype=float32)>,
 <Dense name=dense_1, built=True>)

In [ ]:
# Trainable parameters (weights & baises)
layer.trainable_variables

[<Variable path=dense_1/kernel, shape=(3, 2), dtype=float32, value=[[-0.32881343  0.5527402 ]
  [-0.14014363 -0.3658083 ]
  [-0.9868201   1.0813589 ]]>,
 <Variable path=dense_1/bias, shape=(2,), dtype=float32, value=[0. 0.]>]

In [27]:
with tf.GradientTape() as tape:
  # Forward pass
  y = layer(x)
  loss = tf.reduce_mean(y**2)

grad = tape.gradient(loss, layer.trainable_variables)

In [28]:
loss, np.mean(np.square([0.       , 3.0652003]))

(<tf.Tensor: shape=(), dtype=float32, numpy=4.697726726531982>,
 4.697726439560045)

In [29]:
grad

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.       , 3.0652003],
        [0.       , 6.1304007],
        [0.       , 9.1956005]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.       , 3.0652003], dtype=float32)>]

### Graphs and tf.function

TensorFlow originally used **static computation graphs** (TF 1.x), where the computation is described first, and then run.
In TensorFlow 2.x, the default mode is **eager execution**, which is more intuitive (like PyTorch) but slightly less performant. To get the performance of static graphs, you use:`@tf.function` This decorator converts Python functions into TensorFlow computation graphs.

In [45]:
# Coverting python fun to Tensorflow graph

@tf.function
def get_MSE(y_true, y_pred):
    sq_diff = tf.pow((y_true-y_pred),2)
    print("Calculating MSE")
    return tf.reduce_mean(sq_diff)


In [46]:
y_true = tf.random.uniform([5],maxval=10)
y_pred = tf.random.uniform([5],maxval=10)

print(y_true)
print(y_pred)

tf.Tensor([8.35292   9.274664  5.5500736 5.202558  6.934885 ], shape=(5,), dtype=float32)
tf.Tensor([7.318878  2.5516403 9.539125  5.304489  5.4756594], shape=(5,), dtype=float32)


In [ ]:
##However, tf.function can behave differently under graph and eager execution. 
# The Python print function is one example of how these two modes differ. 
# Tracing captures the TensorFlow operations into a graph, and print is not captured in the graph. That graph is then executed for all three calls without ever running the Python code again.

error = get_MSE(y_true, y_pred) 
error = get_MSE(y_true, y_pred)# It won't print beacauce earlier alredy function traced. When function traced then only prints
error = get_MSE(y_true, y_pred)


Calculating MSE


In [49]:
tf.config.run_functions_eagerly(True)
get_MSE(y_true, y_pred)

Calculating MSE


<tf.Tensor: shape=(), dtype=float32, numpy=12.864109992980957>

In [50]:
# Don't forget to set it back when you are done.
tf.config.run_functions_eagerly(False)

In [54]:
## Checking Speed with without tf.function

def matrix_power(x, y):
    result = tf.eye(len(x), dtype=tf.dtypes.int32)

    for _ in range(y):
        result = tf.matmul(x, result)
    
    return result

In [55]:
x = tf.random.uniform([10, 10], maxval=2, dtype=tf.dtypes.int32)
x

<tf.Tensor: shape=(10, 10), dtype=int32, numpy=
array([[0, 1, 1, 1, 0, 1, 1, 0, 0, 1],
       [0, 0, 1, 1, 1, 1, 0, 1, 1, 0],
       [1, 1, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [1, 0, 0, 0, 1, 1, 1, 0, 1, 1]])>

In [ ]:
#without tf function
# %time
# matrix_power(x, 1000)

print("Eager execution:", timeit.timeit(lambda: matrix_power(x, 100), number=1000), "seconds")

Eager execution: 3.413141999975778 seconds


In [62]:
matrix_power_as_graph = tf.function(matrix_power)
print("Graph execution:", timeit.timeit(lambda: matrix_power_as_graph(x, 100), number=1000), "seconds")

Graph execution: 0.8467953000217676 seconds
